# The Model 

Code by Hmrishav Bandyopadhyay
https://towardsdatascience.com/real-time-object-detection-pytorch-yolo-f7fec35afb64

In [ ]:
import os
from lib.train_AI_lib import *

In [ ]:
os.chdir('..')
print(os.getcwd())

In [ ]:
import cv2
#### Filters we want to use ####
edgeDetect  = lambda oImg: cv2.Canny(oImg, 125, 200)
edgeDetect2 = lambda fImg: cv2.Canny(fImg, 175, 250)
laplacian   = lambda pImg: cv2.Laplacian(cv2.cvtColor(pImg, cv2.COLOR_BGR2GRAY),cv2.CV_64F)
gradient    = lambda gImg: cv2.morphologyEx(cv2.cvtColor(gImg, cv2.COLOR_BGR2GRAY), cv2.MORPH_GRADIENT, (5,5))
opening     = lambda cImg: cv2.morphologyEx(cv2.cvtColor(cImg, cv2.COLOR_BGR2GRAY), cv2.MORPH_OPEN, kernel)
bilateral   = lambda bImg: cv2.bilateralFilter(cv2.cvtColor(bImg, cv2.COLOR_BGR2GRAY),9,100,100)

In [ ]:
torch.cuda.is_available()

In [ ]:
import torch
from torch import  nn

#o = [i + 2*p - k - (k-1)*(d-1)]/s + 1--formula to calculate padding


class YoloNet(nn.Module):

    #YOLO model
    '''
    Input size of the model is 
    448x448x3
    In tensor notation, expressed as [batchsize,3,448,448]
    output--
    [batchsize,30,7,7]
    '''

    def __init__(self,name):
        super(YoloNet,self).__init__()
        self.name = name

        self.t1=nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=64,kernel_size=(7,7),stride=2,padding=(2,2)),
            nn.MaxPool2d(kernel_size=(2,2),stride=2),
        )
        
        self.t2=nn.Sequential(
            nn.Conv2d(in_channels=64,out_channels=192,kernel_size=(3,3),padding=(1,1)),
            nn.MaxPool2d(kernel_size=(2,2),stride=2),
        )
        self.t3=nn.Sequential(
            nn.Conv2d(in_channels=192,out_channels=128,kernel_size=(1,1)),
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=(3,3),padding=(1,1)),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=(1,1)),
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=(3,3),padding=(1,1)),
            nn.MaxPool2d(kernel_size=(2,2),stride=2),
        )

        self.t4=nn.Sequential(
            nn.Conv2d(in_channels=512,out_channels=256,kernel_size=(1,1)),
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=(3,3),padding=(1,1)),
            nn.Conv2d(in_channels=512,out_channels=256,kernel_size=(1,1)),
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=(3,3),padding=(1,1)),
            nn.Conv2d(in_channels=512,out_channels=256,kernel_size=(1,1)),
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=(3,3),padding=(1,1)),
            nn.Conv2d(in_channels=512,out_channels=256,kernel_size=(1,1)),
            nn.Conv2d(in_channels=256,out_channels=512,kernel_size=(3,3),padding=(1,1)),
            nn.Conv2d(in_channels=512,out_channels=512,kernel_size=(1,1)),
            nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=(3,3),padding=(1,1)),

            nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=(3,3),stride=2)
        )

        self.t5=nn.Sequential(
            nn.Conv2d(in_channels=1024,out_channels=512,kernel_size=(1,1)),
            nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=(3,3),padding=(1,1)),
            nn.Conv2d(in_channels=1024,out_channels=512,kernel_size=(1,1)),
            nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=(3,3),padding=(1,1)),

            nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=(3,3),padding=(1,1)),

            nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=(3,3),stride=2,padding=(1,1))

        )

        self.t6=nn.Sequential(
            nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=(3,3),padding=(1,1)),
            nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=(3,3),padding=(1,1))
        )


        
    def forward(self,x):
        x=self.t1(x)
        x=self.t2(x)
        x=self.t3(x)
        x=self.t4(x)
        x=self.t5(x)
        x=self.t6(x)
        
        x=torch.flatten(x,1)
        x=nn.Linear(x.size()[1],4096)(x)
        x=nn.Linear(4096,7*7*30)(x)
        x=x.view(-1,30,7,7)

        return x #output of model

In [ ]:
trainLoader, valLoader, testLoader = loadData(batchsize = 32, args = {'func': edgeDetect2})

In [ ]:
my_model = CNN(name = "edgeDetect2")
use_cuda = True

if use_cuda and torch.cuda.is_available():
  my_model.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')

trainNet(net = my_model,
         data = [trainLoader, valLoader], 
         batchsize = 32, 
         epochNo = 15, 
         lr = 0.001)

In [ ]:
dir = "/content/drive/My Drive/OpenCV Model/edgedetect2_b64_te10_lr0.001"
if not os.path.exists(dir):
    os.mkdir(dir)

In [ ]:
#net = exNetClass('netA'); net.cuda()
netPath = '/content/drive/My Drive/OpenCV Model/edgedetect2_b64_te10_lr0.001'
#net.load_state_dict(torch.load(netPath+'/model_epoch9'))
trainLoader, valLoader, testLoader = loadData(batchsize = 1, args = {'func': edgeDetect2})
_ = regresAnalysis(my_model, trainLoader, netPath)